In [12]:
from MADLens import *
import json
from vmad.lib import fastpm
import numpy as np

In [15]:
from MADLens.lightcone import WLSimulation

In [5]:
params_file = '/home/nessa/Documents/codes/MADLens/runs/cd8f0a9/deriv_run0.json'

In [6]:
with open(params_file, 'r') as f:
    params = json.load(f)

In [7]:
print(params)

{'logtostderr': False, 'alsologtostderr': False, 'log_dir': '', 'v': 0, 'verbosity': 0, 'stderrthreshold': 'fatal', 'showprefixforinfo': True, 'run_with_pdb': False, 'pdb_post_mortem': False, 'run_with_profiling': False, 'profile_file': None, 'use_cprofile_for_profiling': True, 'only_check_args': False, 'results_path': '/home/nessa/Documents/codes/MADLens/results/cd8f0a9', 'PGD_path': '/home/nessa/Documents/codes/MADLens/pgd_params/', 'N_maps': 1, 'boxsize': 256.0, 'Nmesh': [16, 16, 16], 'Nmesh2D': [32, 32], 'boxsize2D': 6.37616, 'N_steps': 11, 'custom_cosmo': False, 'Omega_m': 0.3089, 'sigma_8': 0.8158, 'PGD': False, 'B': 2, 'zs_source': [1.0], 'interpolate': False, 'debug': True, 'save3D': False, 'save3Dpower': False, 'vjp': True, 'jvp': False, 'forward': True, 'analyze': False, 'label': 'deriv_run', '?': False, 'help': False, 'helpshort': False, 'helpfull': False, 'helpxml': False, 'BoxSize': [256.0, 256.0, 256.0], 'BoxSize2D': [6.37616, 6.37616]}


In [8]:
from nbodykit.cosmology import Planck15

In [9]:
cosmo = Planck15.match(Omega0_m=params['Omega_m'])
cosmo = cosmo.match(sigma8=params['sigma_8'])


In [17]:
pm        = fastpm.ParticleMesh(Nmesh=params['Nmesh'], BoxSize=params['BoxSize'], resampler='cic')

    # 2D FOV in radians
BoxSize2D = [deg/180.*np.pi for deg in params['BoxSize2D']]

np.random.seed(123)
randseeds = np.random.randint(0,1e6,100)

    # generate initial conditions
cosmo     = cosmo.clone(P_k_max=30)
rho       = pm.generate_whitenoise(seed=randseeds[0], unitary=False, type='complex')
rho       = rho.apply(lambda k, v:(cosmo.get_pklin(k.normp(2) ** 0.5, 0) / pm.BoxSize.prod()) ** 0.5 * v)
#set zero mode to zero
rho.csetitem([0, 0, 0], 0)

    # weak lensing simulation object
wlsim     = WLSimulation(stages = np.linspace(0.1, 1.0, params['N_steps'], endpoint=True), cosmology=cosmo, pm=pm, boxsize2D=BoxSize2D, params=params)

#build
if params['interpolate']:
    model     = wlsim.run_interpolated.build()
else:
    model     = wlsim.run.build()


number of box replications required to fill field of view: 1.0
insufficient number of rotations to fill lightcone


In [ ]:
model.compute(vout='kmaps', init=dict(rhok=rho))